In [ ]:
!pip install PyPDF2 pdfplumber spacy nltk
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md

In [100]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from IPython.display import IFrame
import pdfplumber
import spacy
import PyPDF2

# Extract text from PDF using pdfplumber
def pdf_to_text_pdfplumber(file_path):
    text = ""    
    with pdfplumber.open(file_path) as pdf:        
        for page in pdf.pages:            
            text += page.extract_text()
    return text

# Extract text from PDF using PyPDF2
def pdf_to_text_pypdf2(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        return text

In [101]:
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")

In [102]:

#file_path = "your_pdf_file_path"
# IFrame(file_path, width=800, height=600)

In [ ]:
#print(nlp.get_pipe("ner").labels)

In [104]:
#text = pdf_to_text_pdfplumber(file_path)
text = pdf_to_text_pypdf2(file_path)
# print(text)

In [ ]:
# Preprocess text: remove stop words, convert to lowercase
stop_words = set(stopwords.words("english"))
tokens = word_tokenize(text.lower())
tokens = [t for t in tokens if t not in stop_words]
doc = nlp(" ".join(tokens))

In [105]:
# Process the text with SpaCy
doc = nlp(text)

In [110]:
# 1. Rule-based Extraction: Look for patterns like "Submitted by:", "Prepared by:"
possible_names_1 = []
for i, token in enumerate(doc):
    if token.text.lower() in ("submitted", "prepared", "response", "input"):
        if doc[i+1].text.lower() == "by":
            possible_names_1.append(" ".join([t.text for t in doc[i+2:i+6] if t.text.isalpha()]))

print(possible_names_1)

[]


In [111]:
# 2. Extract from Header/Footer
possible_names_2 = []
for line in text.split("\n")[:5]:  # Check first 5 lines for header
    possible_names_2.extend([ent.text for ent in nlp(line).ents if ent.label_ == "ORG"])
for line in text.split("\n")[-5:]: # Check last 5 lines for footer
    possible_names_2.extend([ent.text for ent in nlp(line).ents if ent.label_ == "ORG"])

print(possible_names_2)

['the Draft Sports Broadcasting Services Regulations', 'the South African Youth Council']


In [112]:
# 3. Extract from First Few Lines
possible_names_3 = []
possible_names_3.extend([ent.text for ent in nlp(" ".join(text.split("\n")[:10])).ents if ent.label_ == "ORG"])

print(possible_names_3)

['the Draft Sports Broadcasting Services Regulations', 'the South African Youth Council', 'the South African Youth Council']


In [113]:
# 4. Extract from Signature Section (New - Assumes signature is in last 10 lines)
possible_names_4 = []
possible_names_4.extend([ent.text for ent in nlp(" ".join(text.split("\n")[-10:])).ents if ent.label_ == "ORG"])

print(possible_names_4)

['SAYC']


In [114]:
# 5. If no names found using above methods, extract all organizations
possible_names_5 = []
#if not possible_names:
possible_names_5 = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

print(possible_names_5)

['the Draft Sports Broadcasting Services Regulations', 'the South African Youth Council', 'the South African Youth Council', 'SAYC', 'SAYC']


In [99]:
# 6. Find the most similar name to the last few lines, first few lines, and the entire document (Modified)
if possible_names:
    last_lines = " ".join(text.split("\n")[-5:])
    first_lines = " ".join(text.split("\n")[:5:])
    most_similar_name = max(possible_names, key=lambda name: 
                           nlp(name).similarity(nlp(last_lines))
                        + nlp(name).similarity(nlp(first_lines))
#                        + nlp(name).similarity(nlp(text))) 
print(most_similar_name)

the Draft Sports Broadcasting Services Regulations


/var/folders/36/3266hv_j31l96qxrj4pw5cyr0000gn/T/ipykernel_57786/1687954684.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  nlp(name).similarity(nlp(last_lines)) +
/var/folders/36/3266hv_j31l96qxrj4pw5cyr0000gn/T/ipykernel_57786/1687954684.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  nlp(name).similarity(nlp(first_lines)) +
/var/folders/36/3266hv_j31l96qxrj4pw5cyr0000gn/T/ipykernel_57786/1687954684.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  nlp(name).similarity(nlp(text)))
